# 1. Parameters

In [1]:
# Defaults

## Directories
simulation_dir <- "simulations/unset"
reference_file <- "input/S_HeidelbergSL476.fasta.gz"
initial_tree_file <- "input/salmonella.tre"
reference_name <- "NC_011083"

## Simulation parameters
sub_lambda <- 1e-2
sub_mu <- 1
sub_invariant <- 0.9
ins_rate <- 1e-4
ins_max_length <- 60
ins_a <- 1.6
del_rate <- 1e-4
del_max_length <- 60
del_a <- 1.6

## Read simulation information
read_coverage <- 5
read_length <- 250

## Other
ncores <- 48

In [2]:
# Parameters
read_coverage = 40
mincov = 10
simulation_dir = "simulations/cov-40"
iterations = 3


In [3]:
output_dir <- file.path(simulation_dir, "simulated_data")
output_vcf_prefix <- file.path(output_dir, "haplotypes")
reads_data_initial_prefix <- file.path(output_dir, "reads_initial", "data")
print(output_dir)
print(output_vcf_prefix)

[1] "simulations/cov-40/simulated_data"


[1] "simulations/cov-40/simulated_data/haplotypes"


# 2. Generate simulated data

This simulates *Salmonella* data using a reference genome and a tree.

In [4]:
library(jackalope)

# Make sure we've complied with openmp
jackalope:::using_openmp()

[1] TRUE

In [5]:
reference <- read_fasta(reference_file)
reference$set_names(c(reference_name))
reference_len <- 4888768 # I don't know how to programatically get this in R
reference

< Set of 1 chromosomes >
# Total size: 4,888,768 bp
  name                             chromosome                             length
NC_011083  AGAGATTACGTCTGGTTGCAAGAGATCA...ATAACGTGCTGTAATTTTTAAAATAATA   4888768

In [6]:
library(ape)

tree <- read.tree(initial_tree_file)
tree <- root(tree, "reference", resolve.root=TRUE)
tree


Phylogenetic tree with 60 tips and 59 internal nodes.

Tip labels:
  reference, SH12-013, SH12-014, SH10-015, SH10-014, SH11-002, ...
Node labels:
  Root, 1.000000, 0.000000, 0.000000, 1.000000, 0.000000, ...

Rooted; includes branch lengths.

In [7]:
sub <- sub_JC69(lambda = sub_lambda, mu = sub_mu, invariant = sub_invariant)
ins <- indels(rate = ins_rate, max_length = ins_max_length, a = ins_a)
del <- indels(rate = del_rate, max_length = del_max_length, a = del_a)

ref_haplotypes <- create_haplotypes(reference, haps_phylo(tree), sub=sub, ins=ins, del=del)
ref_haplotypes

                              << haplotypes object >>
# Haplotypes: 60
# Mutations: 220,974

                          << Reference genome info: >>
< Set of 1 chromosomes >
# Total size: 4,888,768 bp
  name                             chromosome                             length
NC_011083  AGAGATTACGTCTGGTTGCAAGAGATCA...ATAACGTGCTGTAATTTTTAAAATAATA   4888768

# 3. Write simulated data

In [8]:
write_vcf(ref_haplotypes, out_prefix=output_vcf_prefix, compress=TRUE)

In [9]:
assemblies_prefix = file.path(output_dir, "assemblies", "data")

write_fasta(ref_haplotypes, out_prefix=assemblies_prefix,
            compress=TRUE, n_threads=ncores, overwrite=TRUE)

In [10]:
n_samples <- length(tree$tip)
n_reads <- round((reference$sizes() * read_coverage * n_samples) / read_length)
print(sprintf("Number of reads for coverage %sX and read length %s over %s samples with respect to reference with length %s: %s", 
              read_coverage, read_length, n_samples, reference_len, n_reads))

illumina(ref_haplotypes, out_prefix = reads_data_initial_prefix, sep_files=TRUE, n_reads = n_reads,
         frag_mean = read_length * 2 + 50, frag_sd = 100,
         compress=TRUE, comp_method="bgzip", n_threads=ncores,
         paired=TRUE, read_length = read_length)

[1] "Number of reads for coverage 40X and read length 250 over 60 samples with respect to reference with length 4888768: 46932173"


In [11]:
# Remove the simulated reads for the reference genome since I don't want these in the tree
ref1 <- paste(toString(reads_data_initial_prefix), "_reference_R1.fq.gz", sep="")
ref2 <- paste(toString(reads_data_initial_prefix), "_reference_R2.fq.gz", sep="")
if (file.exists(ref1)) {
    file.remove(ref1)
    print(sprintf("Removing: %s", ref1))
}
if (file.exists(ref2)) {
    file.remove(ref2)
    print(sprintf("Removing: %s", ref2))
}

[1] "Removing: simulations/cov-40/simulated_data/reads_initial/data_reference_R1.fq.gz"


[1] "Removing: simulations/cov-40/simulated_data/reads_initial/data_reference_R2.fq.gz"


In [12]:
# Remove the new reference assembly genome since I don't need it
ref1 <- paste(toString(assemblies_prefix), "__reference.fa.gz", sep="")
if (file.exists(ref1)) {
    file.remove(ref1)
    print(sprintf("Removing: %s", ref1))
}

[1] "Removing: simulations/cov-40/simulated_data/assemblies/data__reference.fa.gz"
